# Import these functions

In [ ]:
!pip install yahoo_fin
!pip install yoptions

In [5]:
import concurrent.futures

# Fundemental & Technical Data
import yfinance as yf 

# Options Data
import yoptions as yo
from yahoo_fin import options

# General Helper Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta



File not found.


In [19]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Optionally, set the max rows displayed in the output as well
pd.set_option('display.max_rows', 100)

## Technical Features (Technical Indicators known to be reliable for ML and forecasting)

In [20]:
def calculate_ema(data, period):
    alpha = 2 / (period + 1)
    return data.ewm(alpha=alpha, adjust=False).mean()

def calculate_dema(data, period):
    ema1 = calculate_ema(data, period)
    ema2 = calculate_ema(ema1, period)
    return 2 * ema1 - ema2


# Define a function to compute the moving averages according to the type.
def moving_average(df, ma_type, period):
    if ma_type == "DEMA":
        return calculate_dema(df, period)
    elif ma_type == "EMA":
        return calculate_ema(df, period)
    elif ma_type == "SMA":
        return df.rolling(window=period).mean()


In [21]:
def macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def rsi(data, periods=14, ema=True):
    close_delta = data['Close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    if ema:
        ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    else:
        ma_up = up.rolling(window=periods).mean()
        ma_down = down.rolling(window=periods).mean()
    rsi = ma_up / ma_down
    data['RSI'] = 100 - (100 / (1 + rsi))
    return data


def obv(data):
    data['OBV'] = np.where(data['Close'] > data['Close'].shift(1), data['Volume'],
                           np.where(data['Close'] < data['Close'].shift(1), -data['Volume'], 0)).cumsum()
    return data

def atr(data, window=14):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    data['ATR'] = true_range.rolling(window=window).mean()
    return data

def stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    data['%K'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

In [22]:
def woodie_pivots(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    pivot = (high + low + 2 * close) / 4
    data['Pivot'] = pivot
    data['R1'] = 2 * pivot - low
    data['S1'] = 2 * pivot - high
    data['R2'] = pivot + (high - low)
    data['S2'] = pivot - (high - low)
    data['R3'] = high + 2 * (pivot - low)
    data['S3'] = low - 2 * (high - pivot)
    data['R4'] = pivot + 3 * (high - low)
    data['S4'] = pivot - 3 * (high - low)
    return data

In [24]:
def bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
    data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
    return data

# Calculate the predictive rolling mean and standard deviation
def predict_bollinger_bands(series, period, num_std, shift_periods):
    rolling_mean = series.rolling(window=period).mean()
    rolling_std = series.rolling(window=period).std()

    # Predict the future values by shifting the mean and std
    predicted_mean = rolling_mean.shift(-shift_periods)
    predicted_std = rolling_std.shift(-shift_periods)

    # Calculate the upper and lower predictive bands
    predicted_upper_band = predicted_mean + (predicted_std * num_std)
    predicted_lower_band = predicted_mean - (predicted_std * num_std)

    return predicted_upper_band, predicted_lower_band

# Calculate the predictive bands for 3 periods ahead

# Calculate the rolling mean and standard deviation for the Bollinger Bands
def calculate_bollinger_bands(series, period, num_std):
    rolling_mean = series.rolling(window=period).mean()
    rolling_std = series.rolling(window=period).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, rolling_mean, lower_band

# Calculate predictive Bollinger Bands by shifting the bands forward
def calculate_predictive_bands(series, period, num_std, shift_periods):
    upper_band, middle_band, lower_band = calculate_bollinger_bands(series, period, num_std)
    # Shift the calculated bands forward by the shift_periods
    predictive_upper_band = upper_band.shift(periods=shift_periods)
    predictive_lower_band = lower_band.shift(periods=shift_periods)
    # For the predicted SMA, simply use the shifted rolling mean
    predictive_middle_band = middle_band.shift(periods=shift_periods)
    return predictive_upper_band, predictive_middle_band, predictive_lower_band



In [25]:
############################################################################################################
# Process other non-stationary data
############################################################################################################
# Function to detrend time series data using a linear regression model
def detrend_data(data, column):
    # Linear regression model requires reshaped index as a feature
    X = np.arange(len(data)).reshape(-1, 1)
    y = data[column].values  # Original values to detrend
    
    # Create and fit the model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict the trend
    trend = model.predict(X)
    
    # Detrend by subtracting the trend from the original data
    detrended = y - trend
    data[f'{column}_detrended'] = detrended
    
    # Return the detrended data and the trend for further analysis
    return data, trend

def seasonal_decomposition(data, column, period):
    # Perform seasonal decomposition
    decomposition = seasonal_decompose(data[column], model='multiplicative', period=period)
    
    # Add components to DataFrame
    data['trend_component'] = decomposition.trend
    data['seasonal_component'] = decomposition.seasonal
    data['residual_component'] = decomposition.resid
    
    # Seasonally adjust the data
    data[column + '_seasonally_adjusted'] = data[column] / data['seasonal_component']
    
    return data

# Function to calculate price differences
def calculate_price_differences(data, column):
    data[f'{column}_diff'] = data[column].diff()
    return data

# Function to calculate log returns
def calculate_log_returns(data, column):
    data[f'{column}_log_return'] = np.log(data[column] / data[column].shift(1))
    return data

# Function to calculate volume changes
def calculate_volume_changes(data, volume_column):
    data[f'{volume_column}_changes'] = data[volume_column].diff()
    return data


## Data Preparation Helpers

In [28]:

def fetch_fundamentals(ticker):
    """
    Fetches comprehensive fundamental data for a given ticker, including balance sheet and cash flow,
    and returns it as a DataFrame.

    Args:
    - ticker (str): The ticker symbol of the stock.

    Returns:
    - DataFrame: DataFrame with merged market, technical, and fundamental data.
    """
    try:
        # Define start date and end date based on current date and one year ago
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

        ticker_obj = yf.Ticker(ticker)
        
        # Fetch Beta from ticker's info
        beta_value = ticker_obj.info.get('beta', 0)
        
        balance_sheet = ticker_obj.balance_sheet
        cashflow = ticker_obj.cashflow

        balance_sheet_transposed = balance_sheet.T
        cashflow_transposed = cashflow.T

        fundamentals = pd.concat([balance_sheet_transposed, cashflow_transposed], axis=1)
        fundamentals.index.names = ['Date']
        
        # Insert Beta as the first column
        fundamentals.insert(0, 'Beta', beta_value)

        fundamentals.fillna(method='backfill', inplace=True)
        fundamentals.fillna(method='ffill', inplace=True)
        fundamentals.fillna(0, inplace=True)

        # Example of calculating growth rate of free cash flows (replace with your actual data)
        free_cash_flows = pd.Series([100, 120, 140, 160, 180])
        growth_rate = free_cash_flows.pct_change().mean()
        print("Free Cash Flow Growth Rate:", growth_rate)

        return fundamentals

    except Exception as e:
        print(f"Failed to fetch or process fundamental data for {ticker}: {e}")
        return pd.DataFrame()  # Return empty DataFrame in case of failure

# Apply the function to fetch and merge fundamental data for MSFT
fundamental_data = fetch_fundamentals('MSFT')

# Displaying the first few rows of the fundamental dataset
fundamental_data


Free Cash Flow Growth Rate: 0.15863095238095237


C:\Users\Administrator\AppData\Local\Temp\ipykernel_2944\1286849980.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2944\1286849980.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundamentals.fillna(method='backfill', inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2944\1286849980.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fundamentals.fillna(method='ffill', inplace=True)


,Beta,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Gains Losses Not Affecting Retained Earnings,Other Equity Adjustments,Retained Earnings,Capital Stock,Common Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Other Non Current Liabilities,Tradeand Other Payables Non Current,Non Current Deferred Liabilities,Non Current Deferred Revenue,Non Current Deferred Taxes Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Long Term Debt,Current Liabilities,Other Current Liabilities,Current Deferred Liabilities,Current Deferred Revenue,Current Debt And Capital Lease Obligation,Current Debt,Pensionand Other Post Retirement Benefit Plans Current,Payables And Accrued Expenses,Payables,Total Tax Payable,Income Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Investments And Advances,Long Term Equity Investment,Goodwill And Other Intangible Assets,Other Intangible Assets,Goodwill,Net PPE,Accumulated Depreciation,Gross PPE,Leases,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Properties,Current Assets,Other Current Assets,Hedging Assets Current,Inventory,Finished Goods,Work In Process,Raw Materials,Receivables,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Issuance Of Capital Stock,Capital Expenditure,End Cash Position,Beginning Cash Position,Effect Of Exchange Rate Changes,Changes In Cash,Financing Cash Flow,Cash Flow From Continuing Financing Activities,Net Other Financing Charges,Cash Dividends Paid,Common Stock Dividend Paid,Net Common Stock Issuance,Common Stock Payments,Common Stock Issuance,Net Issuance Payments Of Debt,Net Short Term Debt Issuance,Net Long Term Debt Issuance,Long Term Debt Payments,Long Term Debt Issuance,Investing Cash Flow,Cash Flow From Continuing Investing Activities,Net Other Investing Changes,Net Investment Purchase And Sale,Sale Of Investment,Purchase Of Investment,Net Business Purchase And Sale,Purchase Of Business,Net PPE Purchase And Sale,Purchase Of PPE,Operating Cash Flow,Cash Flow From Continuing Operating Activities,Change In Working Capital,Change In Other Working Capital,Change In Other Current Liabilities,Change In Other Current Assets,Change In Payables And Accrued Expense,Change In Payable,Change In Account Payable,Change In Inventory,Change In Receivables,Changes In Account Receivables,Stock Based Compensation,Deferred Tax,Deferred Income Tax,Depreciation Amortization Depletion,Depreciation And Amortization,Depreciation,Operating Gains Losses,Gain Loss On Investment Securities,Net Income From Continuing Operations
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-06-30,0.89,7.432000e+09,7.432000e+09,1.253300e+10,5.996500e+10,1.289710e+11,2.534600e+11,8.010800e+10,1.289710e+11,1.272800e+10,2.062230e+11,2.482130e+11,2.062230e+11,2.062230e+11,-6.343000e+09,-6.343000e+09,1.188480e+11,9.371800e+10,9.371800e+10,2.057530e+11,1.016040e+11,1.798100e+10,2.556000e+10,3.345000e+09,2.912000e+09,433000000.0,5.471800e+10,1.272800e+10,4.199000e+10,1.041490e+11,1.474500e+10,5.090100e+10,5.090100e+10,5.247000e+09,5.247000e+09,1.100900e+10,2.224700e+10,2.224700e+10,4.152000e+09,4.152000e+09,1.809500e+10,4.119760e+11,2.277190e+11,3.060100e+10,9.879000e+09,9.879000e+09,7.725200e+10,9.366000e+09,6.788600e+10,1.099870e+11,-6.825100e+10,1.782380e+11,8.537000e+09,1.434600e+10,8.120700e+10,6.846500e+10,5.68

In [27]:
def fetch_technical_data(ticker='SPY', start_year=1993, end_year=None, interval='1d', calculate_indicators=False, include_fundamentals=False, export_csv=False, csv_file=None,):
    """
    Fetches data for a specified ticker from Yahoo Finance from the given start year to the current year or specified end year at specified intervals.
    
    Parameters:
        ticker (str): The ticker symbol for the asset. Defaults to 'SPY'.
        start_year (int): The year from which to start fetching the data. Defaults to 1993.
        end_year (int): The last year for which to fetch the data. Defaults to the current year if None.
        interval (str): The data interval ('1d' for daily, '1wk' for weekly, '1mo' for monthly, '1h' for hourly).
        export_csv (bool): Whether to export the data to a CSV file. Defaults to False.
        csv_file (str): The path of the CSV file to export the data to. Automatically determined if None.
        calculate_indicators (bool): Flag to calculate technical indicators.
        include_fundamentals (bool): Flag to include fundamental data.

    Returns:
        DataFrame: DataFrame containing the requested financial data.
    """
    # Adjust for hourly data to limit to the last 730 days
    if interval == '1h':
        start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')
    else:
        start_date = f'{start_year}-01-01'
    
    if end_year is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    else:
        end_date = f"{end_year}-12-31"
    
    if csv_file is None:
        csv_file = f'{ticker}_{interval}_data_{start_date}_to_{end_date}.csv'
    
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)
    
    if not data.empty:
        if calculate_indicators:
            # Here you would call your indicator functions on the `data` DataFrame
            data = bollinger_bands(data)
            data = macd(data)
            data = rsi(data)
            data = woodie_pivots(data)
            data = obv(data)
            data = atr(data)
            data = stochastic_oscillator(data)

            # Non-stationary data processing
            data = calculate_price_differences(data, 'Close')  # Calculate price differences
            data = calculate_log_returns(data, 'Close')  # Calculate log returns for the 'Close' column
            data = calculate_volume_changes(data, 'Volume')  # Calculate volume changes
            # Handling NaN values by forward filling then dropping rows with NaN values
            data.ffill(inplace=True)
            data.dropna(inplace=True)
    return data

MSFT = fetch_technical_data('MSFT', interval="1d", calculate_indicators=True, include_fundamentals=True)

MSFT.head()

,Open,High,Low,Close,Adj Close,Volume,Bollinger_High,Bollinger_Low,MACD,Signal,RSI,Pivot,R1,S1,R2,S2,R3,S3,R4,S4,OBV,ATR,%K,%D,Close_diff,Close_log_return,Volume_changes
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-01-29,2.734375,2.750000,2.679688,2.703125,1.672746,39424000,2.864005,2.658652,0.008243,0.021679,43.908854,2.708984,2.738281,2.667969,2.779297,2.638672,2.808593,2.597657,2.919921,2.498048,-96377600,0.077846,18.918785,17.355565,-0.015625,-0.005764,-37606400.0
1993-02-01,2.695313,2.750000,2.671875,2.734375,1.692085,42854400,2.857140,2.672939,0.006781,0.018700,49.769723,2.722656,2.773438,2.695312,2.800781,2.644531,2.851562,2.617188,2.957031,2.488281,-53523200,0.077846,29.729614,24.324199,0.031250,0.011494,3430400.0
1993-02-02,2.730469,2.796875,2.726563,2.781250,1.721092,70371200,2.856504,2.681387,0.009298,0.016819,57.023650,2.771484,2.816406,2.746094,2.841797,2.701172,2.886718,2.675782,2.982421,2.560548,16848000,0.073940,45.945857,31.531419,0.046875,0.016998,27516800.0
1993-02-03,2.796875,2.820313,2.750000,2.761719,1.709005,71728000,2.855385,2.680552,0.009606,0.015377,53.553385,2.773438,2.796875,2.726562,2.843751,2.703125,2.867188,2.656250,2.984377,2.562499,-54880000,0.071149,58.000020,44.558497,-0.019531,-0.007047,1356800.0
1993-02-04,2.742188,2.742188,2.640625,2.656250,1.643739,124214400,2.863732,2.665175,0.001325,0.012566,39.554622,2.673828,2.707031,2.605469,2.775391,2.572265,2.808594,2.503906,2.978517,2.369139,-179094400,0.070312,8.695629,37.547168,-0.105469,-0.038938,52486400.0


# Data Preprocessing

In [ ]:
"""
calculate_returns(data, periods): Calculates returns over specified periods (e.g., 1, 2, 3 months) for given stock prices.
calculate_volatility(data, periods): Computes volatility over specified periods for given stock prices.
calculate_moving_averages(data, periods): Calculates moving averages over specified periods for given stock prices.
calculate_MA_gaps(data, periods): Computes moving average gaps over specified periods to capture trends.
apply_log_transform(data, columns): Applies logarithmic transformation to specified columns to normalize data.
add_lag_features(data, columns, lags): Adds lagged features for specified columns and lag periods to capture time-series dependencies.
"""

# Advanced Feature Engineering

In [ ]:
"""
encode_cyclical_features(data, column, max_val): Encodes cyclical features like days of the week or months in a year using sine and cosine transformations to preserve their cyclical nature.
calculate_technical_indicators(data, indicators): Calculates a range of technical indicators (e.g., RSI, MACD, Bollinger Bands) specified by the user for given stock prices.
apply_PCA_reduction(data, n_components): Applies PCA (Principal Component Analysis) to reduce the dimensionality of the feature space while retaining n_components principal components.
"""


# Trading Strategy Simulation and Evaluation

In [ ]:
"""
simulate_trading_strategy(predictions, transaction_costs): Simulates a trading strategy based on model predictions and calculates net profit after accounting for transaction costs.
calculate_max_drawdown(returns): Calculates the maximum drawdown from a series of returns, useful for risk assessment.
backtest_strategy(time_series, strategy_function): Conducts a backtest of a trading strategy function on a historical time series dataset to evaluate performance over time.
"""



# Model Training and Evaluation




In [ ]:
"""
train_test_split_time_series(data, test_ratio): Splits time-series data into training and testing sets with a specified ratio, preserving the temporal order.
cross_validate_time_series(model, data, cv_splits): Performs time-series cross-validation with a given model, data, and number of splits.
evaluate_model_performance(model, X_test, y_test, metrics): Evaluates the model performance using specified metrics (e.g., RMSE, MAE, Pearson correlation).
hyperparameter_optimization(model, param_grid, X_train, y_train): Conducts hyperparameter optimization for a given model and parameter grid.
"""



# Prediction and Ranking

In [ ]:
"""
predict_and_rank(model, data, features): Generates predictions using a trained model and ranks the predictions for trading strategies.
adjust_predictions_based_on_median(data, securities_to_adjust): Dynamically adjusts predictions based on the median target value for specific securities.
calculate_spread_return_sharpe(data, portfolio_size, weight_ratio): Calculates the Sharpe ratio based on spread return for a given portfolio size and weight ratio.
"""

# Data Visualization

In [ ]:
"""
plot_stock_prices(data, title): Plots stock prices or returns over time.
plot_feature_importances(model, feature_names): Plots the feature importances for a trained model.
plot_prediction_vs_actual(data, predictions, actual, title): Plots predicted vs. actual values for visual comparison.
"""


# Data Integrity and Cleaning

In [ ]:
"""
detect_outliers(data, method): Detects outliers in the dataset using a specified method (e.g., IQR, Z-score).
impute_missing_values(data, imputation_strategy): Imputes missing values using a specified strategy (e.g., mean, median, k-NN).
"""


# 

# 